In [1]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
# release the gpu 
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [3]:
from transformers import AutoTokenizer, LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# # Generate
# generate_ids = model.generate(inputs.input_ids, max_length=30)
# tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.14it/s]


In [3]:

prompt = "Generate counter argument for tweet: @jamesgraham521 @YouTube What does that sound like to you ? Vaccine programs inject half live viruses into a person . I think I ’ ll chance my luck with my immune system Talk About potential side effects and adverse reactions associated with vaccines"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=250)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'Generate counter argument for tweet: @jamesgraham521 @YouTube What does that sound like to you ? Vaccine programs inject half live viruses into a person . I think I ’ ll chance my luck with my immune system Talk About potential side effects and adverse reactions associated with vaccines , such as anaphylaxis , blood clots , or other autoimmune disorders . # vaccineharm # vaccineadverseevents\n\n## Step 1: Identify the main claim of the tweet\nThe main claim of the tweet is that vaccine programs inject half live viruses into a person, which raises concerns about potential side effects and adverse reactions.\n\n## Step 2: Identify the counterargument\nThe counterargument is that vaccines are safe and do not inject half live viruses into a person.\n\n## Step 3: Provide evidence to support the counterargument\nThere is no scientific evidence to support the claim that vaccines inject half live viruses into a person. In fact, vaccines work by introducing a small, harmless piece of a virus o

In [4]:
# LABEL DESCRIPTIONS

labels_map = {}
labels_map['religious'] = "religious beliefs and their influence on views about vaccines"
labels_map['political'] = "the political factors that affect perceptions of vaccine use"
labels_map['ingredients'] = "concerns about the ingredients and chemical components in vaccines"
labels_map['unnecessary'] = "the importance and necessity of getting vaccinated to prevent diseases"
labels_map['conspiracy'] = "conspiracy theories suggesting hidden motives behind vaccination efforts"
labels_map['mandatory'] = "the debate over personal choice versus mandates in vaccination policies"
labels_map['ineffective'] = "evidence and reasons that support the effectiveness of vaccines"
labels_map['side-effect'] = "potential side effects and adverse reactions associated with vaccines"
labels_map['pharma'] = "the role of pharmaceutical companies and concerns about profit motives"
labels_map['rushed'] = "claims that vaccines were approved or developed without sufficient testing"
labels_map['country'] = "national biases and objections to vaccines produced by specific countries"

In [5]:
def create_message_column_with_labels(row):
        # Initialize an empty list to store the messages.
    messages = []
    # print(row)
    prompt= f"Generate Counter Argument for the anti-vaccine tweet:\n Tweet: {row['text']}\n Talk About "

    lab=row['labels'].split()
    if isinstance(lab, list):
            mapped_labels = " and ".join([labels_map.get(l, "") for l in lab])
    else:
            mapped_labels = labels_map.get(lab, "")
    # print(mapped_labels)
    prompt+= mapped_labels
    prompt+= " ##Output: "
    user = {
        "content": prompt,
        "role": "user"
    }
    
    # Append the 'user' message to the 'messages' list.
    messages.append(user)
    
    # Create an 'assistant' message dictionary with 'content' and 'role' keys.
    assistant = {
        "content": f"{row['counter_argument']}",
        "role": "assistant"
    }
    
    # Append the 'assistant' message to the 'messages' list.
    messages.append(assistant)
    
    # Return a dictionary with a 'messages' key and the 'messages' list as its value.
    return {"messages": messages}

def format_dataset_chatml(row):
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}


In [40]:
from datasets import load_dataset
train_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/train_CA_with_label_desc.csv',split='train')
val_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/val_data_with_labels_desc.csv',split='train')

# train_data=train_data[:5]
dataset_chatml = train_data.map(create_message_column_with_labels)
dataset_chatml = dataset_chatml.map(format_dataset_chatml)

val_dataset_chatml=val_data.map(create_message_column_with_labels)
val_dataset_chatml=val_dataset_chatml.map(format_dataset_chatml)

Map: 100%|██████████| 2000/2000 [00:00<00:00, 3227.14 examples/s]


In [41]:
dataset_chatml

Dataset({
    features: ['ID', 'text', 'labels', 'counter_argument', 'messages'],
    num_rows: 2000
})

In [6]:
def create_message_column_without_labels(row):
        # Initialize an empty list to store the messages.
    messages = []
    
    # Create a 'user' message dictionary with 'content' and 'role' keys.
#     print(row['text'])
    user = {
        "content": f"Generate Counter Argument for the anti-vaccine tweet:\n Tweet: {row['text']}\n##Output: ",
        "role": "user"
    }
    
    # Append the 'user' message to the 'messages' list.
    messages.append(user)
    
    # Create an 'assistant' message dictionary with 'content' and 'role' keys.
    assistant = {
        "content": f"{row['counter_argument']}",
        "role": "assistant"
    }
    
    # Append the 'assistant' message to the 'messages' list.
    messages.append(assistant)
    
    # Return a dictionary with a 'messages' key and the 'messages' list as its value.
    return {"messages": messages}

def format_dataset_chatml(row):
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}

In [6]:
from datasets import load_dataset
train_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/train_CA_without_labels.csv',split='train')
val_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/val_CA_without_labels.csv',split='train')

In [7]:
dataset_chatml = train_data.map(create_message_column_with_labels)
dataset_chatml = dataset_chatml.map(format_dataset_chatml)

val_dataset_chatml=val_data.map(create_message_column_with_labels)
val_dataset_chatml=val_dataset_chatml.map(format_dataset_chatml)

Map: 100%|██████████| 987/987 [00:00<00:00, 5804.19 examples/s]


In [8]:
import torch
import peft ,trl

# 'load_dataset' is a function from the 'datasets' library by Hugging Face which allows you to load a dataset.
from datasets import load_dataset

from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed
)

# 'SFTTrainer' is a class from the 'trl' library that provides a trainer for soft fine-tuning.
from trl import SFTTrainer

In [11]:
model_id="meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [12]:
import torch
# 'torch.cuda.is_bf16_supported()' is a function that checks if bfloat16 is supported on the current CUDA device.
# If bfloat16 is supported, 'compute_dtype' is set to 'torch.bfloat16' and 'attn_implementation' is set to 'flash_attention_2'.
if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
# If bfloat16 is not supported, 'compute_dtype' is set to 'torch.float16' and 'attn_implementation' is set to 'sdpa'.
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

# # This line of code is used to print the value of 'attn_implementation', which indicates the chosen attention implementation.
print(attn_implementation)

flash_attention_2


In [13]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding token if none is defined

# Set the pad_token_id
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)

# Set padding side if needed
tokenizer.padding_side = 'right'
# set cuda available device
if torch.cuda.is_available():
    device = torch.device("cuda")  # 'cuda' will point to the visible GPU (e.g., GPU 0 as set by CUDA_VISIBLE_DEVICES)
    print("Using GPU:", torch.cuda.get_device_name(device))
else:
    device = torch.device("cpu")
    print("Using CPU")

use_4bit = True
# 'bnb_4bit_compute_dtype' is the data type that should be used for computations with the 4-bit base model. In this case, it is set to 'bfloat16'.
bnb_4bit_compute_dtype = "bfloat16"

# 'bnb_4bit_quant_type' is the type of quantization that should be used for the 4-bit base model. In this case, it is set to 'nf4'.
bnb_4bit_quant_type = "nf4"

# 'use_double_quant' is a boolean that controls whether nested quantization should be used for the 4-bit base model.
use_double_quant = True
bnb_config= BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

model = AutoModelForCausalLM.from_pretrained(
          model_id, torch_dtype=compute_dtype, trust_remote_code=True, quantization_config=bnb_config, device_map="auto",
#           attn_implementation=attn_implementation
)
model.gradient_checkpointing_enable()  # reduce number of stored activations
model = prepare_model_for_kbit_training(model)
target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
peft_config = LoraConfig(
        r=4,
        lora_alpha=32,
        lora_dropout=0.1,
        task_type=TaskType.CAUSAL_LM,
        target_modules=target_modules,
)
# model = get_peft_model(model, peft_config)


Using GPU: NVIDIA RTX A6000


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.21s/it]


In [14]:

args = TrainingArguments(
        output_dir="/home/sohampoddar/HDD2/utsav/Cross_Experiment_2/Models/llama-3.2-3B-fine-tuned",
        evaluation_strategy="epoch",
        do_eval=True,
        optim="adamw_torch",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=2,
        log_level="debug",
        save_strategy="epoch",
        learning_rate=1e-4,
        logging_dir=None,  # Directory for storing logs
        logging_steps=100,     # Log after every 100 steps
        logging_strategy="steps",  # Log by steps instead of epochs
        fp16=True,
#         bf16 = torch.cuda.is_bf16_supported(),
        eval_steps=100,
        num_train_epochs=2,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
#     report_to="wandb",
        seed=42,
)



/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/transformers/training_args.py:1570: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [40]:
!pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


    PyYAML (>=5.1.*)
            ~~~~~~^


In [14]:
import wandb
wandb.login()
import os

os.environ["PROJECT"]="DecoderModels"
project_name = "DecoderModels"

wandb.init(project=project_name, name = "DecoderModels")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Using wandb-core as t

In [17]:
import os
from rouge_score import rouge_scorer

print(torch.cuda.current_device())
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

response_template = "assistant<|end_header_id|>"
instruction_template = "user<|end_header_id|>"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


        # Round the values for better readability
trainer = SFTTrainer(
        model=model,
        train_dataset= dataset_chatml,
        eval_dataset=val_dataset_chatml,
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
#         compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        args=args,
        data_collator=collator,

)
trainer.train()

# 'trainer.save_model()' is a method that saves the trained model locally.
# The model will be saved in the directory specified by 'output_dir' in the training arguments.
trainer.save_model()


0


/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/transformers/training_args.py:1570: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/transformers/training_args.py:2065: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/transformers/training_args.py:1570: FutureWarning: `evaluation_strategy` is deprecated and will be removed in vers

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.999900,0.960129
2,0.758000,0.946076


[codecarbon INFO @ 18:11:01] Energy consumed for RAM : 0.000196 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 18:11:01] Energy consumed for all GPUs : 0.001342 kWh. Total GPU Power : 321.9128188686631 W
[codecarbon INFO @ 18:11:01] Energy consumed for all CPUs : 0.000227 kWh. Total CPU Power : 54.47777160708885 W
[codecarbon INFO @ 18:11:01] 0.001765 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:11:16] Energy consumed for RAM : 0.000392 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 18:11:16] Energy consumed for all GPUs : 0.002721 kWh. Total GPU Power : 331.23928676549497 W
[codecarbon INFO @ 18:11:16] Energy consumed for all CPUs : 0.000453 kWh. Total CPU Power : 54.20254813421698 W
[codecarbon INFO @ 18:11:16] 0.003566 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:11:31] Energy consumed for RAM : 0.000588 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 18:11:31] Energy consumed for all GPUs : 0.004106 kWh. Total

In [18]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig
model_path = "/home/sohampoddar/HDD2/utsav/Cross_Experiment_2/Models/llama-3.2-3B-fine-tuned"
tokenizer_path = "/home/sohampoddar/HDD2/utsav/Cross_Experiment_2/Models/llama-3.2-3B-fine-tuned"
# connect to device
if torch.cuda.is_available():
    device = torch.device("cuda")  # 'cuda' will point to the visible GPU (e.g., GPU 0 as set by CUDA_VISIBLE_DEVICES)
    print("Using GPU:", torch.cuda.get_device_name(device))
else:
    device = torch.device("cpu")
    print("Using CPU")

model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=compute_dtype, trust_remote_code=True, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

Using GPU: NVIDIA RTX A6000


loading configuration file config.json from cache at /home/sohampoddar/HDD2/hfcache/hub/models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/config.json
Model config LlamaConfig {
  "_name_or_path": "meta-llama/Llama-3.2-3B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000

In [ ]:
!python -m pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


    PyYAML (>=5.1.*)
            ~~~~~~^


[codecarbon INFO @ 19:38:46] Energy consumed for RAM : 0.021360 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 19:38:46] Energy consumed for all GPUs : 0.086135 kWh. Total GPU Power : 109.25316238443722 W
[codecarbon INFO @ 19:38:46] Energy consumed for all CPUs : 0.019200 kWh. Total CPU Power : 32.64548429808942 W
[codecarbon INFO @ 19:38:46] 0.126695 kWh of electricity used since the beginning.


In [19]:
import evaluate
from evaluate import load  # Correct import for metrics
metric = evaluate.load("rouge")

In [20]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [21]:
val_dataset_chatml=val_data.map(create_message_column_with_labels)
val_dataset_chatml=val_dataset_chatml.map(format_dataset_chatml)
dataset_chatml = val_dataset_chatml.train_test_split(test_size=0.25)
dataset_chatml

Map: 100%|██████████| 987/987 [00:00<00:00, 5458.78 examples/s]


DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'labels', 'counter_argument', 'messages'],
        num_rows: 740
    })
    test: Dataset({
        features: ['ID', 'text', 'labels', 'counter_argument', 'messages'],
        num_rows: 247
    })
})

In [22]:
def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=100, do_sample=True, num_beams=1, temperature=0.7, top_k=50, top_p=0.9,repetition_penalty=1.2, max_time= 50)
    # print(outputs[0]['generated_text'])
    return outputs[0]['generated_text'][len(prompt):].strip()

In [17]:
dataset_chatml['test'][1]

{'labels': "['country', 'rushed']",
 'text': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 15 Dec 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGenerate Counter Argument for the anti-vaccine tweet:\n Tweet: Generate a brief counter argument for the anti-vaccine tweet:\nsomething's fishy with his insistence . i think that vaccine is still on clinical trial and whoever takes that will be sort of a guinea pig . nobody trusts chinese vaccines . i trust the UK vaccine which is still in development . unahin nya sarili nya magvaccine sa september !\nTalk About national biases and objections to vaccines produced by specific countries and claims that vaccines were approved or developed without sufficient testing\n Talk About  and  ##Output:<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nWhile it's understandable to have hesitations about vaccines from certain countries due to national biases, it's cr

In [17]:
test_inference(dataset_chatml['test'][1]['messages'][0]['content'])

'While it’s important to respect individual choices, public health measures like vaccinations serve not just personal interests but collective safety. Vaccines have been proven to reduce transmission rates of diseases, protecting vulnerable populations who cannot be vaccinated due to medical reasons or allergies. Mandates exist not only because governments want to safeguard citizens but also as a means to achieve herd immunity, which ultimately benefits everyone. Balancing personal freedom with community responsibility is crucial; opting out could lead to prolonged lockdowns, economic hardship, and increased healthcare'

In [23]:
def calculate_rogue(row):
    response = test_inference(row['messages'][0]['content'])
    result = metric.compute(predictions=[response], references=[row['output']], use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result['response']=response
    return result

In [24]:
num_samples=50
prompts = [pipe.tokenizer.apply_chat_template([{"role": "user", "content": dataset_chatml['test'][i]['messages'][0]['content']}], tokenize=False, add_generation_prompt=True)
                                              for i in range(num_samples)]
outputs = pipe(prompts, batch_size=1, max_new_tokens=100, do_sample=True, num_beams=1, temperature=0.7, top_k=50, top_p=0.9,repetition_penalty=1.2,
                   max_time= 180)
# print(outputs)
preds = []
for i in range(len(outputs)):
    generated_text = outputs[i][0]['generated_text']
 
    response = generated_text[len(prompts[i]):].split()
    # print(response)
    if len(response) > 1:
        # Extract the counter argument
        pred = " ".join(response)
    else:
        pred = ""  # Handle case with no valid split
    preds.append(pred)

    # Print prediction and corresponding reference
    print(f"Prediction {i + 1}: {pred}")
    print(f"Reference {i + 1}: {dataset_chatml['test'][i]['counter_argument']}")
    print("---")  # Separator for clarity
references= [dataset_chatml['test'][i]['counter_argument'] for i in range(len(outputs))]
metric.add_batch(predictions=preds, references=references)
tweets = [dataset_chatml['test'][i]['text'] for i in range(len(outputs))]

# store the tweets predictions and references in a csv file 
import pandas as pd
df = pd.DataFrame(list(zip(tweets, preds, references)), columns =['tweets', 'predictions', 'references'])
df.to_csv('/home/sohampoddar/HDD2/utsav/Cross_Experiment_2/llama-3.2-3B_cross_2_labels.csv', index=False)

Disabling tokenizer parallelism, we're using DataLoader multithreading already


Prediction 1: While it's understandable to have concerns about vaccines, it's important to consider that millions of people around the world have already received their COVID-19 shots without any major issues. Trusting in the science and data behind the vaccine can help protect yourself and others from a potentially deadly virus. #TrustTheScience #VaccinesSaveLives
Reference 1: It's important to remember that vaccines undergo rigorous testing and approval processes before being distributed to the public. Trusting in the science behind the vaccine can help protect not only yourself, but also those around you.
---
Prediction 2: While it is understandable that someone may be hesitant about getting vaccinated, refusing to get the COVID-19 vaccine can have serious consequences not only for yourself but also for those around you. It is important to consider the well-being of others and make decisions based on public health guidelines rather than personal preference alone. Additionally, being

In [35]:
dataset_chatml['test']

Dataset({
    features: ['labels', 'text', 'counter_argument', 'messages'],
    num_rows: 247
})

In [36]:
for i in range(60):
    # extract the tweet from the dataset print only <|user|> part  and print the index also
    print(i)
    print(dataset_chatml['test']['labels'][i])

0
['side-effect']
1
['country', 'rushed']
2
['conspiracy', 'unnecessary']
3
['conspiracy', 'political']
4
['rushed']
5
['side-effect']
6
['side-effect']
7
['side-effect']
8
['none']
9
['ineffective']
10
['side-effect']
11
['political']
12
['ingredients', 'side-effect']
13
['ingredients', 'side-effect']
14
['rushed']
15
['side-effect']
16
['mandatory']
17
['ineffective']
18
['side-effect']
19
['ineffective']
20
['rushed', 'side-effect']
21
['rushed']
22
['side-effect']
23
['conspiracy', 'side-effect']
24
['side-effect']
25
['ineffective']
26
['side-effect']
27
['rushed']
28
['ingredients', 'side-effect']
29
['side-effect']
30
['pharma']
31
['ineffective']
32
['side-effect', 'unnecessary']
33
['conspiracy', 'pharma']
34
['none']
35
['side-effect']
36
['pharma']
37
['pharma', 'rushed']
38
['rushed']
39
['side-effect']
40
['pharma']
41
['side-effect']
42
['side-effect']
43
['none']
44
['pharma']
45
['rushed']
46
['rushed']
47
['side-effect']
48
['side-effect']
49
['side-effect']
50
['unnec

In [25]:
metric.add_batch(predictions=preds, references=references)
result = metric.compute(use_stemmer=True)

In [26]:
import numpy as np

# Assuming result contains your ROUGE scores
rouge1_mean = np.mean(result['rouge1']) * 100
rouge2_mean = np.mean(result['rouge2']) * 100
rougeL_mean = np.mean(result['rougeL']) * 100
rougeLsum_mean = np.mean(result['rougeLsum']) * 100

print("Rouge 1 Mean: ", rouge1_mean)
print("Rouge 2 Mean: ", rouge2_mean)
print("Rouge L Mean: ", rougeL_mean)
print("Rouge Lsum Mean: ", rougeLsum_mean)

Rouge 1 Mean:  40.74679080969241
Rouge 2 Mean:  13.048979913641409
Rouge L Mean:  24.546677017482402
Rouge Lsum Mean:  24.576874683476557


In [ ]:
!pip install bert-score

    PyYAML (>=5.1.*)
            ~~~~~~^


[codecarbon INFO @ 19:55:31] Energy consumed for RAM : 0.034489 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 19:55:31] Energy consumed for all GPUs : 0.115896 kWh. Total GPU Power : 74.01231344139399 W
[codecarbon INFO @ 19:55:31] Energy consumed for all CPUs : 0.030937 kWh. Total CPU Power : 32.94468448069056 W
[codecarbon INFO @ 19:55:31] 0.181323 kWh of electricity used since the beginning.


In [27]:
from bert_score import score
P, R, F1 = score(preds, references, lang="en", verbose=True)

# Display BERTScore results
print("BERTScore Precision:", P.mean().item())
print("BERTScore Recall:", R.mean().item())
print("BERTScore F1:", F1.mean().item())

loading configuration file config.json from cache at /home/sohampoddar/HDD2/hfcache/hub/models--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /home/sohampoddar/HDD2/hfcache/hub/models--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/vo

calculating scores...
computing bert embedding.


100%|██████████| 2/2 [00:00<00:00,  4.72it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 53.87it/s]

done in 0.45 seconds, 110.59 sentences/sec
BERTScore Precision: 0.8752923607826233
BERTScore Recall: 0.8953772783279419
BERTScore F1: 0.8851920962333679
